In [14]:
from typing import List
import databases
import sqlalchemy
from pydantic import BaseModel
import os
import urllib

In [15]:
 


#DATABASE_URL = "sqlite:///./test.db"

host_server = os.environ.get('host_server', 'localhost')
db_server_port = urllib.parse.quote_plus(str(os.environ.get('db_server_port', '5432')))
database_name = os.environ.get('database_name', 'alecore')
db_username = urllib.parse.quote_plus(str(os.environ.get('db_username', 'alecore')))
db_password = urllib.parse.quote_plus(str(os.environ.get('db_password', 'ale_S@cr@t')))
ssl_mode = urllib.parse.quote_plus(str(os.environ.get('ssl_mode','prefer')))
DATABASE_URL = 'postgresql://{}:{}@{}:{}/{}?sslmode={}'.format(db_username, db_password, host_server, db_server_port, database_name, ssl_mode)

database = databases.Database(DATABASE_URL)

In [16]:
metadata = sqlalchemy.MetaData()

notes = sqlalchemy.Table(
    "user",
    metadata,
    sqlalchemy.Column("id", sqlalchemy.Integer, primary_key=True),
    sqlalchemy.Column("pass", sqlalchemy.String),
    sqlalchemy.Column("completed", sqlalchemy.Boolean),
)

engine = sqlalchemy.create_engine(
    #DATABASE_URL, connect_args={"check_same_thread": False}
    DATABASE_URL, pool_size=3, max_overflow=0
)
metadata.create_all(engine)

In [17]:
class NoteIn(BaseModel):
    text: str
    completed: bool

class Note(BaseModel):
    id: int
    text: str
    completed: bool

In [188]:
d = pd.read_pickle('/home/ifte/alechat_core/core/components/model/output/features_index.pkl')

In [18]:
from databases import Database 
import asyncio 
  
async def create_table(): 
  
    database = databases.Database(DATABASE_URL)
    try: 
        await database.connect() 
        print('Connected to Database') 
  
        # Create a table. 
        query = """CREATE TABLE usage (id INTEGER PRIMARY KEY, name VARCHAR(100))"""
        print('Created Table usage created Successfully') 
        await database.execute(query=query) 
      
        await database.disconnect() 
        print('Disconnecting from Database') 
    except : 
        print('Connection to Database Failed') 
  
          
if __name__=='__main__': 
    asyncio.run(create_table())

Connected to Database
Created Table usage created Successfully
Disconnecting from Database


In [21]:
from time import time

import aiopg
import asyncio

CONN_INFO = {
    'host': 'localhost',
    'user': 'alecore',
    'port': 5432,
    'database': 'alecore',
}

dsn = DATABASE_URL


async def get_data(pool):
    start = time()
    async with pool.acquire() as conn:
        async with conn.cursor() as cur:
            await cur.execute('SELECT * FROM usr limit 100000')
            result = await cur.fetchall()
    print(f'there are {len(result)} records. Exec time: {time() - start}')
    return result


async def main():
    pool = await aiopg.create_pool(dsn)

    start = time()
    tasks = []

    for i in range(3):
        tasks.append(loop.create_task(get_data(pool)))

    tasks, stat = await asyncio.wait(tasks)

    for task in tasks:
        print(f'number of items: {len(task.result())}')

    print(f'total exec time: {time() - start} secs')

    print('exiting main')


loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

print('exiting program')

there are 1 records. Exec time: 0.012930631637573242
there are 1 records. Exec time: 0.013180017471313477
there are 1 records. Exec time: 0.013029336929321289
number of items: 1
number of items: 1
number of items: 1
total exec time: 0.01427769660949707 secs
exiting main


/home/ifte/environments/experiment_env/lib/python3.7/site-packages/aiopg/pool.py:312: ResourceWarning: Unclosed 2 connections in <aiopg.pool.Pool object at 0x7f576c4f3810>
  ResourceWarning)


RuntimeError: Cannot close a running event loop